In [1]:
from selenium import webdriver
from chromedriver_py import binary_path # this will get you the path variable
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:

class langwebbuilder:
    def __init__(self):
        self.svc = webdriver.ChromeService(executable_path='/home/chad/software/chromedriver-linux64/chromedriver')
        self.driver = webdriver.Chrome(service=self.svc)
        
    def getVideoLinks(self,url):
        self.driver.get(url) #
        als = self.driver.find_elements(By.TAG_NAME, "iframe")
        for al in als:
            tt = al.get_attribute('class')
            if tt.find('Isolated')>=0:
                self.driver.switch_to.frame(al)
                qals = self.driver.find_elements(By.XPATH,'*')

                qals = self.driver.find_elements(By.TAG_NAME,'source')
                for qal in qals:
                    ret = qal.get_attribute('data-src')
                return ret

    def closeDriver(self):
        self.driver.close()

    # crawl gc website to get urls for each talk page, 
    #  then crawl each of those pages for the video links
    def crawlGC(self,url):
        url = "https://www.churchofjesuschrist.org/study/general-conference/2023/10?lang=eng"

        svc = webdriver.ChromeService(executable_path='/home/chad/software/chromedriver-linux64/chromedriver')
        op = webdriver.ChromeOptions()
        driver = webdriver.Chrome(service=svc)

        driver.get(url) #
        als = driver.find_elements(By.TAG_NAME, "a")
        urls = []

        for al in als:
            tt = al.get_attribute('class')
            href = al.get_attribute('href')
            if href.split('/')[-1][0].isnumeric():
                urls.append(href)

        driver.close()

        videourls = []
        for url in urls:
            x = self.getVideoLinks(url)
            videourls.append({'vidurl':x,'pageurl':url}) # urls for video, and for the talk page
        self.closeDriver()

        self.videourls = videourls

    # make button to open video and page in iframes on click
    def makeButtonScript(self,urlVideo,urlPage,frameVideo,framePage,name,lang):
        Name = name.title()
        ret = """
    <button id="update {name} {lang}">{Name} ({lang})</button>
    <script>
            document.getElementById('update {name} {lang}').addEventListener('click', function () {{
                document.getElementById('{frameVideo}').src ='{urlVideo}';
                document.getElementById('{framePage}').src ='{urlPage}';
            }});
    </script>"""
        ret = ret.format(urlVideo=urlVideo,urlPage=urlPage,
                        frameVideo=frameVideo,framePage=framePage,
                        name=name,lang=lang,Name=Name)
        return ret
        


    def makeHeader(self):
        ret = """
    <!DOCTYPE html>
    <html>

    <body>
        <h1 style="display:flex;
            justify-content:center;color:green;">
            General Conference Language Study
        </h1>

        <iframe src="about:blank" id="frameVideo" width="600px" height="600px"> </iframe>
        <iframe src="about:blank" id="framePage" width="1200px" height="600px"> </iframe>
        """
        return ret+'\n'

    def makeButtons(self):
        videourls = self.videourls
        ret = ""
        for d in videourls:
            ret = ret+"<p>\n"
            vurl = d['vidurl']
            purl = d['pageurl']
            vid = vurl.split('eng.mp4')[0]+"{lang}.mp4"
            page = purl.split('=')[0]+"={lang}"
            for lang in ['eng','spa','ita','deu']:
                vlang = vid.format(lang=lang)
                plang = page.format(lang=lang)
                name = ' '.join(vurl.split('/')[-1].split('-')[3:6])
                r = self.makeButtonScript(vlang,plang,"frameVideo","framePage",name,lang)
                ret = ret+'\n'+r

        return ret+'\n'

    def makeFooter(self):
        ret = "</body>\n</html>\n"
        return ret

    def buildWebsite(self,name):
        videourls = self.crawlGC('x')
        h = self.makeHeader()
        b = self.makeButtons()
        f = self.makeFooter()

        out = open(name,'w')
        out.write(h)
        out.write(b)
        out.write(f)
        out.close()

In [3]:
zz = langwebbuilder()
zz.buildWebsite('index.html')